# Homework 4

This Notebook builds on the DCOPF model introduced in [Notebook 6](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks) and incorporates some elements of Economic Dispatch introduced in [Notebook 4](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks).

First, load (or install if necessary) a set of packages you'll need for this assignment...

In [1]:
using JuMP
using HiGHS
using DataFrames
using CSV
using Plots; plotly();

## Question 1: Modifying IEEE-14

**A. Increased generation costs**

Copy the IEEE 14 bus system and DCOPF solver function from Notebook 6. In addition, add the following line to the return call of the function:
```julia
status = termination_status(DCOPF)
```
This tells you the solver termination status for the problem: e.g. was an optimal solution found, was the solution infeasible, was it unbounded, etc.

Make the following change to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh

Run the DCOPF and output generation, flows, and prices.

In [33]:
datadir = joinpath("..","Notebooks","ieee_test_cases") 
gens = CSV.read(joinpath(datadir,"Gen14.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load14.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters
lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.01938,0.05917,1,10000,1,16.9005
2,1,5,0.05403,0.22304,1,10000,2,4.4835
3,2,3,0.04699,0.19797,1,10000,3,5.05127
4,2,4,0.05811,0.17632,1,10000,4,5.67151
5,2,5,0.05695,0.17388,1,10000,5,5.75109
6,3,4,0.06701,0.17103,1,10000,6,5.84693
7,4,5,0.01335,0.04211,1,10000,7,23.7473
8,4,7,0.0,0.20912,1,10000,8,4.78194
9,4,9,0.0,0.55618,1,10000,9,1.79798


In [34]:
lines.b = lines.reactance ./ (lines.reactance.^2 + lines.resistance.^2);
lines.g = lines.resistance ./ (lines.reactance.^2 + lines.resistance.^2);
lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b,g
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,1,2,0.01938,0.05917,1,10000,1,15.2631,4.99913
2,1,5,0.05403,0.22304,1,10000,2,4.23498,1.0259
3,2,3,0.04699,0.19797,1,10000,3,4.78186,1.13502
4,2,4,0.05811,0.17632,1,10000,4,5.11584,1.68603
5,2,5,0.05695,0.17388,1,10000,5,5.19393,1.70114
6,3,4,0.06701,0.17103,1,10000,6,5.06882,1.98598
7,4,5,0.01335,0.04211,1,10000,7,21.5786,6.84098
8,4,7,0.0,0.20912,1,10000,8,4.78194,0.0
9,4,9,0.0,0.55618,1,10000,9,1.79798,0.0


In [35]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[N]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])

    # Flow constraints on each branch; 
    # In DCOPF, line flow is a function of voltage angles
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineFlows = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineFlows[l] = @constraint(DCOPF, 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
        )
    end
    
    # Max line flow limits
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineLimits = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineLimits[l] = @constraint(DCOPF,
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
        ) 
    end

    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data) #
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end
gens

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,1,0.0430293,20,0,300,0,100,-100,161.62,1
2,2,0.25,25,0,140,0,100,-100,97.47,2


In [36]:
gens[1,3] = 30
gens

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,1,0.0430293,30,0,300,0,100,-100,161.62,1
2,2,0.25,25,0,140,0,100,-100,97.47,2


In [37]:
solution = dcopf_ieee(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
46 rows, 47 cols, 140 nonzeros
41 rows, 42 cols, 130 nonzeros
Presolve : Reductions: rows 41(-55); columns 42(-182); elements 130(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -5.1736490962e+00 Pr: 41(743849); Du: 0(7.01557e-12) 0s
         42     7.0700000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 42
Objective value     :  7.0700000000e+03
HiGHS run time      :          0.02


In [7]:
solution.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


In [8]:
solution.cost

7070.000000000007

In [9]:
solution.generation

Row,node,gen
,Int64,Float64
1,1,119.0
2,2,140.0


In [10]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,62.9289
2,1,5,56.0711
3,2,3,74.3555
4,2,4,59.5201
5,2,5,47.3532
6,3,4,-19.8445
7,4,5,-54.3229
8,4,7,29.355
9,4,9,16.8436


Regarding the above results, answer the following:

- How has generation changed compared to the default system?
- What explains the new prices?


In [11]:
All nodes are at the higher variable price of $30/MWh - gen1 has become the marginal plant

LoadError: syntax: extra token "nodes" after end of expression

**B. Constrained line**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW

Run the DCOPF and output generation, flows, and prices.

In [17]:
lines[3,6] =72
lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b,g
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,1,2,0.01938,0.05917,1,10000,1,15.2631,4.99913
2,1,5,0.05403,0.22304,1,10000,2,4.23498,1.0259
3,2,3,0.04699,0.19797,1,72,3,4.78186,1.13502
4,2,4,0.05811,0.17632,1,10000,4,5.11584,1.68603
5,2,5,0.05695,0.17388,1,10000,5,5.19393,1.70114
6,3,4,0.06701,0.17103,1,10000,6,5.06882,1.98598
7,4,5,0.01335,0.04211,1,10000,7,21.5786,6.84098
8,4,7,0.0,0.20912,1,10000,8,4.78194,0.0
9,4,9,0.0,0.55618,1,10000,9,1.79798,0.0


In [18]:
solution = dcopf_ieee(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
46 rows, 47 cols, 140 nonzeros
41 rows, 42 cols, 130 nonzeros
Presolve : Reductions: rows 41(-55); columns 42(-182); elements 130(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.9410112891e+00 Pr: 41(714065); Du: 0(7.01557e-12) 0s
         43     7.4867290573e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 43
Objective value     :  7.4867290573e+03
HiGHS run time      :          0.00


In [19]:
solution.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,25.0
3,3,127.142
4,4,56.6022
5,5,48.0202
6,6,50.7154
7,7,55.1812
8,8,55.1812
9,9,54.4337


In [20]:
solution.generation

Row,node,gen
,Int64,Float64
1,1,202.346
2,2,56.6542


In [21]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,132.399
2,1,5,69.9468
3,2,3,72.0
4,2,4,54.6227
5,2,5,40.7306
6,3,4,-22.2
7,4,5,-61.1799
8,4,7,29.1034
9,4,9,16.6992


Regarding the above results, answer the following:

- Which node has the highest price and why?
- What is the difference in prices across $l_{23}$, also known as the congestion rent? How do you interpret this value (what is it's practical meaning?)

**C. Demand increase**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW
- Increase demands everywhere by 5\%.

In [41]:
loads[:, 2] = loads[:,2]*1.05 #dataframe has been altered going forward
loads

Row,connnode,demand
,Int64,Float64
1,2,-22.785
2,3,-98.91
3,4,-50.19
4,5,-7.98
5,6,-11.76
6,9,-30.975
7,10,-9.45
8,11,-3.675
9,12,-6.405


Calculate the total available generating capacity:

In [25]:
gensTotal = sum(gens[:,5])

440

Calculate the new total demand:

In [31]:
demTotal = sum(loads)

-271.94999999999993

Run the DCOPF and show prices.

In [42]:
solution = dcopf_ieee(gens, lines, loads);
solution.prices

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
46 rows, 47 cols, 140 nonzeros
41 rows, 42 cols, 130 nonzeros
Presolve : Reductions: rows 41(-55); columns 42(-182); elements 130(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -5.1736490962e+00 Pr: 41(743867); Du: 0(7.01557e-12) 0s
         41     7.4585000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 41
Objective value     :  7.4585000000e+03
HiGHS run time      :          0.00


Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


**What is happening in this system?** 

In [47]:
demand and 

Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


## Question 2: Linear losses

Up until now, we have ignored transmission losses. A quadratic approximation of losses is given by:

\begin{align}
LOSS_{ij} &\approx \frac{G_{ij}}{BaseMVA} (\theta_i-\theta_j)^2 \\
 & \approx \frac{1}{BaseMVA} \frac{R_{ij}}{R_{ij}^2+X_{ij}^2}(\theta_i-\theta_j)^2
\end{align}


where $G$ is the line's conductance, $R$ is the line's resistance and $X$ is the line's reactance. See the `lines` data frame for these parameters.

For our purposes, we will approximate this quadratic via:


$$
LOSS_{ij} \geq \frac{R_{ij}}{BaseMVA} \times (MaxFlow_{ij})^2 
\left(\frac{|FLOW_{ij}|}{MaxFlow_{ij}} - 0.165 \right)
$$

where $MaxFlow_{ij}=200 MW$ in this problem. Note the greater than equal sign, as we do not want to have negative losses.

This approximation is based on Fitiwi et al. (2016), "Finding a representative network losses model for large-scale transmission expansion planning with renewable energy sources," *Energy* 101: 343-358, https://doi.org/10.1016/j.energy.2016.02.015. 

Note that this is a linear approximation of transmission losses, which are actually a quadratic function of power flows. Fitiwi et al. 2016 and other papers describe piece-wise or segment-wise linear approximations of the quadratic function which provide a tighter lower bound approximation of losses, but we'll use a single linear term for this assignment. 

See Jenkins & Sepulveda et al. 2017, "Enhanced decision support for a changing electricity landscape: the GenX configurable electricity resource capacity expansion model", MIT Energy Initiative Working Paper 2017-10 http://bit.ly/GenXModel Section 5.8, for an example of a linear segment-wise approximation of quadratic transmission losses. 


**A. Code linear losses**

Reload the original data from Notebook 6 and copy the IEEE 14 bus system and DCOPF solver function from Notebook 6 into a new function `dcopf_ieee_lossy`.

Make the following changes:
- Increase the variable cost of Generator 1 to \$30 / MWh
- Change all transmission line capacities to 200 MW

Implement losses into the supply/demand balance equations. A standard way to implement absolute values in linear programming is by introducing two non-negative auxiliary variables $x^+$, $x^-$ $\geq 0$:

$$
x = x^+ - x^-
$$

and the absolute value can be represented as:

$$
|x| = x^+ + x^-
$$

(You should satisfy yourself that this equality holds.)

It makes the formulation easier if losses are added to the supply/demand balance constraint in each node by splitting losses in half between the receiving and sending end.

Indicate which equations and variables you have added and explain your steps using inline code comments (e.g. `# Comment`).

Run the lossy DCOPF and output generation, flows, losses, and prices.

**B. Interpret results**

Run the same parameters in the lossless OPF from problem 1. How do prices and flows change? What is the largest magnitude difference in prices between the solution with losses and the lossless OPF solution?

## Question 3 - Security contingencies

Power system operators need to ensure that power is delivered reliably even in the event of unexpected outages (**contingencies**). One common contigency that must be planned for is the loss of a transmission line. The security-constrained OPF (SCOPF) run by operators solves for an optimal dispatch that is simultaneously robust (i.e., feasible) to each of the lines failing individually. This is what is known as **N-1 security**, because we assume that at most one component fails in any given scenario.

In this problem, we will not code a full SCOPF, but rather investigate what happens to the feasibility of our problem when we remove transmission lines.

**A. Setup data**

The following code loads the original dataset (with one row per line) and includes a function `format_lines` that converts this to a format that our solver function can use (duplicating rows for both directions, adding susceptance, etc.).

In [ ]:
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
rename!(lines,lowercase.(names(lines)))

function format_lines(lines)
    # create line ids 
    lines.id = 1:nrow(lines);
    # add set of rows for reverse direction with same parameters
    lines2 = copy(lines)
    lines2.f = lines2.fromnode
    lines2.fromnode = lines.tonode
    lines2.tonode = lines2.f
    lines2 = lines2[:,names(lines)]
    append!(lines,lines2)

    # calculate simple susceptance, ignoring resistance as earlier 
    lines.b = 1 ./ lines.reactance
    return(lines)
end

Next:

1. Set the capacity of all lines in the system at 100 MW, except for the line $l_{12}$, which you should set to 200 MW.

2. Create a load dataframe `loads_sens` that increases demands everywhere by 10\%

**B. Loop over line contingencies**

Create a dataframe `status` with the `fromnode` and `tonode` columns of `lines`.

Create a [for loop](https://docs.julialang.org/en/v1/manual/control-flow/#man-loops) that iterates over each line and:
- sets the reactance to be a very high value, 1e9 (i.e., no power will be transmitted)
- creates a version of the dataframe that our solver function can use via `format_lines`
- runs DCOPF
- stores the solution status in a `opf` column in the corresponding row of the `status` dataframe

Show the `status` results.

**3. Interpret results**

Are all of the cases feasible? If not, how many are infeasible? 

Pick two cases where the solution gives a different status. (For our purposes, dual infeasible and primal infeasible are the same.) What is happening here?

Given this, do you conclude that the system with the assumed transmission line ratings is secure as-is, or do we need to add more redundancy to the system?